In [1]:
!pip install pandas
!pip install music21

  Using cached pandas-2.0.1-cp310-cp310-macosx_11_0_arm64.whl (10.8 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached numpy-1.24.3-cp310-cp310-macosx_11_0_arm64.whl (13.9 MB)

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached music21-8.3.0-py3-none-any.whl (22.8 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached matplotlib-3.7.1-cp310-cp310-macosx_11_0_arm64.whl (7.3 MB)
  Using cached jsonpickle-3.0.1-py2.py3-none-any.whl (40 kB)
  Using cached chardet-5.1.0-py3-none-any.whl (199 kB)
  Using cached more_itertools-9.1.0-py3-none-any.whl (54 kB)
  Using cached kiwisolver-1.4.4-cp310-cp310-macosx_11_0_arm64.whl (63 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.39.4-py3-none-any.whl (1.0 MB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached Pillow-9.5

In [2]:
import pickle
import pandas as pd
import music21
from datetime import datetime, timedelta

In [3]:
piece_id_counter = 1
phrase_id_counter = 1

In [4]:
data = []

files = ["data_alsace"]
for file in files:
    with open('../data/data_2/{}.pkl'.format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

In [5]:
df = pd.DataFrame(columns=['PhraseID','Note','TimeStamp','PieceID'])
for piece in data:
    piece_id = "PIECE{:05}".format(piece_id_counter)
    timestamp = datetime.now()

    for i in range(len(piece[0])):
        if piece[0][i] == 128:
            note = "rest"
        else:
            note = music21.note.Note(piece[0][i]).nameWithOctave
        duration = piece[1][i]
        phrase_id = "PHRASE{:06}".format(phrase_id_counter)
        timestamp += timedelta(seconds = 10 * int(duration))

        new_item = pd.DataFrame(data = {'PhraseID' : [phrase_id], 'Note' : [note], 'TimeStamp' : [timestamp.strftime("%d/%m/%y %H:%M:%S")], "PieceID" : [piece_id] })
        df = pd.concat([df,new_item],ignore_index=True)
        if piece[2][i] == 1:
            timestamp = datetime.now()
            phrase_id_counter += 1
            new_item = pd.DataFrame(data = {'PhraseID' : [phrase_id], 'Note' : 'END', 'TimeStamp' : [timestamp.strftime("%d/%m/%y %H:%M:%S")], "PieceID" : [piece_id] })
            df = pd.concat([df,new_item],ignore_index=True)
        

    piece_id_counter += 1
    if piece_id_counter%1000 == 0:
        df.to_csv("log_data{}.csv".format(piece_id_counter))
        df = pd.DataFrame(columns=['PhraseID','Note','TimeStamp','PieceID'])
    print(piece_id_counter)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


In [ ]:
for piece in data:
    notes = piece[0]
    diff = None
    reset = True 
    for i, p in enumerate(piece[2]):
        if reset:
            diff = music21.note.Note("C4").pitch.midi - notes[i]
            reset = False
        piece[0][i] += diff
        if p == 1:
            reset = True

In [13]:
df = pd.DataFrame(columns=['PhraseID', 'Note', 'PieceID', 'start_timestamp', 'end_timestamp'])

start_time = datetime.combine(datetime.today(), datetime.min.time())

for piece in data:
    piece_id = "PIECE{:05}".format(piece_id_counter)
    timestamp = start_time
    start_timestamp = timestamp

    new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': 'START',
                                  'PieceID': [piece_id], 'start_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                  'end_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")]})
    df = pd.concat([df, new_item], ignore_index=True)
    
    for i in range(len(piece[0])):
        if piece[0][i] == 128:
            note = "rest"
        else:
            note = music21.note.Note(piece[0][i]).nameWithOctave
        duration = piece[1][i]
        phrase_id = "PHRASE{:06}".format(phrase_id_counter)
        timestamp += timedelta(seconds=int(10 * duration))

        new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': [note],
                                      'PieceID': [piece_id], 'start_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                      'end_timestamp': [timestamp.strftime("%d/%m/%y %H:%M:%S")]})
        df = pd.concat([df, new_item], ignore_index=True)

        start_timestamp = timestamp
        
        if piece[2][i] == 1:
            new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': 'END',
                                'PieceID': [piece_id], 'start_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                'end_timestamp': [timestamp.strftime("%d/%m/%y %H:%M:%S")]})
            df = pd.concat([df, new_item], ignore_index=True)
            start_timestamp = start_time
            timestamp = start_time
            phrase_id_counter += 1
            phrase_id = "PHRASE{:06}".format(phrase_id_counter)
            new_item = pd.DataFrame(data={'PhraseID': [phrase_id], 'Note': 'START',
                                'PieceID': [piece_id], 'start_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")],
                                'end_timestamp': [start_timestamp.strftime("%d/%m/%y %H:%M:%S")]})
            df = pd.concat([df, new_item], ignore_index=True)

    piece_id_counter += 1
    
    if piece_id_counter % 1000 == 0:
        df.to_csv("log_data{}.csv".format(piece_id_counter))
        df = pd.DataFrame(columns=['PhraseID', 'Note', 'TimeStamp', 'PieceID', 'start_timestamp', 'end_timestamp'])
    
    #print(piece_id_counter)

2023-05-25 00:00:07
0:00:07
0.75
2023-05-25 00:00:09
0:00:02
0.25
2023-05-25 00:00:14
0:00:05
0.5
2023-05-25 00:00:19
0:00:05
0.5
2023-05-25 00:00:24
0:00:05
0.5
2023-05-25 00:00:26
0:00:02
0.25
2023-05-25 00:00:28
0:00:02
0.25
2023-05-25 00:00:33
0:00:05
0.5
2023-05-25 00:00:05
0:00:05
0.5
2023-05-25 00:00:12
0:00:07
0.75
2023-05-25 00:00:14
0:00:02
0.25
2023-05-25 00:00:19
0:00:05
0.5
2023-05-25 00:00:24
0:00:05
0.5
2023-05-25 00:00:34
0:00:10
1.0
2023-05-25 00:00:39
0:00:05
0.5
2023-05-25 00:00:05
0:00:05
0.5
2023-05-25 00:00:12
0:00:07
0.75
2023-05-25 00:00:14
0:00:02
0.25
2023-05-25 00:00:19
0:00:05
0.5
2023-05-25 00:00:24
0:00:05
0.5
2023-05-25 00:00:29
0:00:05
0.5
2023-05-25 00:00:34
0:00:05
0.5
2023-05-25 00:00:39
0:00:05
0.5
2023-05-25 00:00:05
0:00:05
0.5
2023-05-25 00:00:12
0:00:07
0.75
2023-05-25 00:00:14
0:00:02
0.25
2023-05-25 00:00:19
0:00:05
0.5
2023-05-25 00:00:24
0:00:05
0.5
2023-05-25 00:00:34
0:00:10
1.0
2023-05-25 00:00:39
0:00:05
0.5
2023-05-25 00:00:44
0:00:05
0.

In [14]:
df.to_csv("log_data2.csv")